# How to read data from DataFrames with pyActigraphy

## Imported packages and input data

The usual suspects:

In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import pyActigraphy

In this example, let's generate some input data:

NB: if you already have your data under a pandas.DataFrame format, jump directly to the next section.

In [4]:
N = 1440*7 # 7 days of acquisition at a frequency of 60s.

In [5]:
activity = np.random.normal(10,1,N)
light = np.random.normal(100,10,N)

In [6]:
non_wear = np.empty(N)

In [7]:
# Set up a segment of  spurious inactivity
activity[2060:2160] = 0.0
non_wear[2060:2160] = 1.0

In [8]:
d = {'Activity': activity, 'Light': light, 'Non-wear': non_wear}

In [9]:
index = pd.date_range(start='01-01-2020',freq='60s',periods=N)

In [10]:
data = pd.DataFrame(index=index,data=d)

In [11]:
data

,Activity,Light,Non-wear
2020-01-01 00:00:00,9.291065,93.466639,1.191959e-311
2020-01-01 00:01:00,10.840509,95.308009,1.192555e-311
2020-01-01 00:02:00,10.540116,102.317084,0.000000e+00
2020-01-01 00:03:00,9.182834,114.304216,0.000000e+00
2020-01-01 00:04:00,10.815270,78.546959,0.000000e+00
...,...,...,...
2020-01-07 23:55:00,11.628464,90.688966,0.000000e+00
2020-01-07 23:56:00,7.919542,96.716301,0.000000e+00
2020-01-07 23:57:00,9.032307,101.424161,0.000000e+00
2020-01-07 23:58:00,9.967409,99.999972,0.000000e+00


## Manual creation of a BaseRaw object

In [12]:
from pyActigraphy.io import BaseRaw

help(BaseRaw)

### Set activity and light data (if available)

In [13]:
raw = BaseRaw(
    name="myName", 
    uuid='DeviceId', 
    format='Pandas', 
    axial_mode=None,
    start_time=data.index[0],
    period=(data.index[-1]-data.index[0]),
    frequency=data.index.freq,
    data=data['Activity'],
    light=data['Light']
)

In [14]:
raw.data

2020-01-01 00:00:00     9.291065
2020-01-01 00:01:00    10.840509
2020-01-01 00:02:00    10.540116
2020-01-01 00:03:00     9.182834
2020-01-01 00:04:00    10.815270
                         ...    
2020-01-07 23:55:00    11.628464
2020-01-07 23:56:00     7.919542
2020-01-07 23:57:00     9.032307
2020-01-07 23:58:00     9.967409
2020-01-07 23:59:00     8.219563
Freq: 60s, Name: Activity, Length: 10080, dtype: float64

### Set up a mask

Most devices that have a wear sensor return this information as a binary time series with "1" when the device is most likely not worn and "0" otherwise.
In pyActigraphy, this information can be used to create a mask and thus invalidate the corresponding data points (set to "0" most probably). However, the mask, the value "1" correspond to "no masking". So, depending on your "non-wear" data, be careful to transform them appropriately:

In [15]:
# Here, I assume that 0: the device is worn, 1: device not worn. 
# As mentioned aboce, for the mask, 1: no masking. (NB: it is a convolution: data*mask)
raw.mask = np.abs(data['Non-wear']-1)

## Tests

In [16]:
raw.duration()

<604800 * Seconds>

In [17]:
raw.length()

10080

In [18]:
raw.ADAT(binarize=False)

14229.255750619324

In [19]:
raw.IV(binarize=False)

1.1225988368747062

In [20]:
# If you want to mask the data
raw.mask_inactivity = True

In [21]:
# For a gaussian noise, IV should be close to 2.
raw.IV(binarize=False)

2.2014972524450696

The masking seems to work!

Et voilà!